<a href="https://colab.research.google.com/github/Baronco/Python-LP-Models/blob/main/Python_LP_models_Pulp_%26_Gekko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from pulp import *

# **PULP**

- Una de las ventajas de usar Pulp es que esta libreria permite hacer analisis de precios sombra. Sin embargo, para problemas muy complejos o grandes se tienen problemas de rendimiento serios, pues no se puede hacer uso de arreglos. 

- PulP solo sirve para resolver problemas lineales.

- Con PulP se pueden resolver problemas multiobjetivo

- Las restricciones que se pueden modelar son >= , <= y ==

- X tenis, cada par cuesta 120,000 y las vende en 250,000 
- Y medias, cada par cuesta 2,000 y las venden en 15,000

En un mes, restricciones:

- 800 <= X <= 1200

- 100 <= Y <= 700

- x + y <= 1200

FO: Max ganancias

Z Max = -120,000x + 250,000x -2,000y + 15,000y

Z Max = 130,000x + 13,000y

## **Solucion usando PULP** Caso simple

Esta forma de modelar problemas se puede usar para casos muy sencillos dónde no sea fisicamente imposible definir las variables de decisión una a una

https://coin-or.github.io/pulp/


Restricciones que podemos usar: >=, <=, ==

In [12]:
#Creamos el problema de optimizacion

prob = LpProblem("max_ganancias", LpMaximize)

Tipos de variables a definir:

- Integer

- Binary

- Continuous(default)

In [13]:
#Variables de decision

x = LpVariable("tenis", lowBound = 800, upBound = 1200, cat = 'Integer')
y = LpVariable("medias", lowBound = 100, upBound = 700, cat = 'Integer')

In [14]:
#Otra forma de escribir las restricciones
# prob += x >= 800
# prob += x <= 1200

In [15]:
#Restriccion de maximo de unidades de tenis y medias
prob += x + y <= 1200

In [16]:
#Funcion objetivo
obj = 130000*x + 13000*y

In [17]:
#definir funcion objetivo dentro del problema

prob += obj, "obj"

In [18]:
#Resolvemos
prob.solve()

1

In [61]:
prob

max_ganancias:
MAXIMIZE
13000*medias + 130000*tenis + 0
SUBJECT TO
_C1: medias + tenis <= 1200

VARIABLES
100 <= medias <= 700 Integer
800 <= tenis <= 1200 Integer

1: Solucion encontrada exitosamente!

-1: No hay solucion,no se cumplen restricciones!

In [19]:
print(f'Par de tennis {value(x)}')
print(f'Par de medias {value(y)}')
print(f'Ganancia maxima $ {value(obj):,.0f}')

Par de tennis 1100.0
Par de medias 100.0
Ganancia maxima $ 144,300,000


## **Solucion usando PULP** variables de decision usando diccionarios

Esta forma de modelar los problemas la usamos cuando ya el caso requiera de muchas variables de decision que por motivos de tiempo y esfuerzo nadie escribiria una a una.

In [62]:
#Creamos el problema de optimizacion

prob2 = LpProblem("max_ganancias2", LpMaximize)

In [63]:
#Variables de decision

X = LpVariable.dicts("x",['tenis','medias'],cat = 'Integer')

X

{'tenis': x_tenis, 'medias': x_medias}

In [64]:
#Restricciones
'''
Cuando se usen los diccionarios para simplificar el planteamiento de restriccion todas las variables de ese diccionario deben tener los mismo limites (constrains)

usamos un bucle for
  for i in range(2):
    prob += X[i] <= constrain1
'''

#Como este caso es sencillo no es necesario usar un bucle for
prob2 += X['tenis'] <= 1200
prob2 += X['tenis'] >= 800

prob2 += X['medias'] <= 700
prob2 += X['medias'] >= 100

prob2 += X['tenis'] + X['medias'] <= 1200

A continuación se hacen algunos cambios para que tenga sentido el planteamiento usando diccionarios porque este caso también se le podria plantear de la misma forma la FO

Caso simple:
- obj = 130000 x + 13000 y

Caso con diccionario

- obj = 130000 X['tenis'] + 13000 X['medias']

In [65]:
#coeficientes de la funcion objetivo
'''
Ganancias por par de tenis 130,000
Ganancias por par de medias 13,000
'''
coeficientes = [130000,13000]

In [66]:
#Ejemplo de uso de enumerate
for i,val in enumerate(list(X.keys())):
  print(f'posicion {i}, valor {val}')

posicion 0, valor tenis
posicion 1, valor medias


In [67]:
#Funcion objetivo
obj2 = lpSum([coeficientes[i]*X[val] for i,val in enumerate(list(X.keys()))])

prob2 += obj2

In [68]:
#Resolvemos
prob2.solve()

1

In [69]:
prob2

max_ganancias2:
MAXIMIZE
13000*x_medias + 130000*x_tenis + 0
SUBJECT TO
_C1: x_tenis <= 1200

_C2: x_tenis >= 800

_C3: x_medias <= 700

_C4: x_medias >= 100

_C5: x_medias + x_tenis <= 1200

VARIABLES
x_medias free Integer
x_tenis free Integer

In [74]:
for k,val in X.items():
  print(f'Par de {k}: {value(val)}')
print(f'Ganancia maxima $ {value(obj2):,.0f}')

Par de tenis: 1100.0
Par de medias: 100.0
Ganancia maxima $ 144,300,000


# **Gekko**

Gekko es una libreria más avazanda que permite resolver problemas lineales y no lineales, permite tambien programacion multiobjetivo, dinamica etc.

- La ventaja de usar Gekko para resolver problemas lineales es que este permite modelar usando numpy.arrays, lo cual hace que el procesamiento de la solución sea significativamente superior para problemas complejos. Sin embargo, con Gekko no se puede hacer analisis de precios sombra.

- Las restricciones que se pueden modelar son >=, <= y ==


https://gekko.readthedocs.io/en/latest/

**Compliquemos un poco el tema**

- X tenis, cada par cuesta 120,000 y las venden en 250,000 
- Y medias, cada par cuesta 2,000 y las venden en 15,000
- Z botas, cada par cuesta 80,000 y las venden en 120,000
- W relojs, cada unidad cuesta 100,000 y se vende 200,000

En un mes, restricciones:

- 800 <= X <= 1200

- 100 <= Y <= 700

- 180 <= Z <= 900

- 250 <= W <= 750

- x + y + z + w <= 2500

FO: Max ganancias

Z Max = 130,000x + 13,000y + 40,000z + 100,000w

In [77]:
!pip install gekko

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
from gekko import GEKKO

## **Solucion usando GEKKO** Caso simple

In [108]:
#Inicilizar el problema, remote=False para solucionar en local
problema = GEKKO(remote=False)

In [109]:
#opciones para resolver de forma lineal
problema.options.solver = 1
problema.options.linear = 1

In [110]:
#Definir variables de decision
x = problema.Var(name = 'tenis', lb = 800, ub = 1200,integer=True)
y = problema.Var(name = 'medias', lb = 100, ub = 700, integer=True)
z = problema.Var(name = 'botas', lb = 180, ub = 900, integer=True)
w = problema.Var(name = 'relojes', lb = 250, ub = 750, integer=True)

In [111]:
#Restriccion
problema.Equation(x + y + z + w <= 2500)

In [112]:
#Objectivo
objetivo = 130000*x + 13000*y + 40000*z + 100000*w

problema.Maximize(objetivo)

In [113]:
#Resolvemos, debug=0 permite ver resultados aunque la optimizacion no haya sido exitosa
problema.solve(debug=0)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            5
   Intermediates:            0
   Connections  :            0
   Equations    :            2
   Residuals    :            2
 
 Number of state variables:              5
 Number of total equations: -            1
 Number of slack variables: -            1
 ---------------------------------------
 Degrees of freedom       :              3
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    0 Dpth:    0 Lvs:    0 Obj: -2.50E+08 Gap:  0.00E+00
 Successful solution
 
 ---------------------------------------------------

In [130]:
print(f'Par de tenis: {x.value[0]}')
print(f'Par de medias: {y.value[0]}')
print(f'Par de botas: {z.value[0]}')
print(f'Par de relojes: {w.value[0]}')
print(f'Ganancia maxima $ {130000*x.value[0] + 13000*y.value[0] + 40000*z.value[0] + 100000*w.value[0] :,.0f}')

Par de tenis: 1200.0
Par de medias: 100.0
Par de botas: 450.0
Par de relojes: 750.0
Ganancia maxima $ 250,300,000


## **Solucion usando GEKKO** Caso matricial

In [158]:
#Inicilizar el problema, remote=False para solucionar en local
problema2 = GEKKO(remote=False)

In [159]:
#opciones para resolver de forma lineal
problema2.options.solver = 1
problema2.options.linear = 1

In [160]:
#Definir variables de decision usando vectores
X = problema2.Array(f = problema2.Var,
                    dim = 4,
                    integer = True)

In [161]:
#Restriccion 1
problema2.Equation(X[0] >= 800)
problema2.Equation(X[1] >= 100)
problema2.Equation(X[2] >= 180)
problema2.Equation(X[3] >= 250)

problema2.Equation(X[0] <= 1200)
problema2.Equation(X[1] <= 700)
problema2.Equation(X[2] <= 900)
problema2.Equation(X[3] <= 750)

In [162]:
#Restriccion 2
problema2.Equation(problema.sum(X) <= 2500 )

In [163]:
#Coeficientes ganancias

coef = np.array([130000, 13000, 40000, 100000])

In [164]:
#Funcion objetivo
objetivo2 = np.dot(coef, X)

In [165]:
#Maximizar
problema2.Maximize(objetivo2)

In [166]:
problema2.solve(debug=0)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           13
   Intermediates:            0
   Connections  :            0
   Equations    :           10
   Residuals    :           10
 
 Number of state variables:             13
 Number of total equations: -            9
 Number of slack variables: -            9
 ---------------------------------------
 Degrees of freedom       :             -5
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    0 Dpth:    0 Lvs:    0 Obj: -2.50E+08 Gap:  0.00E+00
 Successful solution
 
 ------------------------------

In [170]:
resultados = []

for i in range(4):
  resultados.append(X[i].value[0])

resultados = np.array(resultados)

print(f'Par de tenis: {resultados[0]}')
print(f'Par de medias: {resultados[1]}')
print(f'Par de botas: {resultados[2]}')
print(f'Par de relojes: {resultados[3]}')

print(f'Ganancia maxima $ {np.dot(coef,resultados) :,.0f}')

Par de tenis: 1200.0
Par de medias: 100.0
Par de botas: 450.0
Par de relojes: 750.0
Ganancia maxima $ 250,300,000
